<a href="https://colab.research.google.com/github/ArshiAbolghasemi/AI-UT/blob/main/knapsack_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Knapsack Problem
In this project we're going to solve our knapsack problem using **Genetic Algorithm**

## Problem Description

Consider that we have a knapsack, and we want to go on a picnic. We have some snacks that we want to take with us. Each snack has a value, and we also have a certain amount of each snack that we can take as much as we want. It's obvious that if we take half of a snack, we will have half its value. Also there is some constraints:
- The sum of the values of our snacks shouldn't exceed a special amount that is given to us.
- The number of snacks that we pick should be within the interval that is given to us.
- The amount of the chosen snack shouldn't exceed the available amount of it.

The snack data is located in snacks.csv in the current directory.

# Settings And Imports

In [2]:
#imports
from typing import List

# Gene And Chromosme
First, we need a definition for our gene and then using that creating a chromosome, that is a potential solution to our problem.

- **Gene**: In the context of this problem, each gene represents an item and the chosen amount. If we don't choose an item, its value is equal to zero; otherwise, it is equal to the amount chosen.

In [1]:
class Gene:

  def __init__(self, _item_name: str, _amount: int):
    self.item_name = _item_name
    self.amount = _amount

  def get_amount(self):
    return self.amount

  def set_amount(self, amount: int):
    self.amount = amount

  def get_item_name(self):
    return self.item_name

- **Chromosome**: A chromosome is a collection of genes that represents a potential solution to the knapsack problem. It encodes a set of items to be included in the knapsack.

In [3]:
class Chromose:

  def __init__(self, genes: List[Gene]):
    self.genes = genes